# CTEs Products Lab 

### Introduction

In this lesson, we'll practice working with CTEs.  As we know CTEs allow to break our queries into multiple steps by creating a temporary table.  And we perform our CTEs with the following syntax:

```SQL
WITH table_name AS (
   SELECT ...
)

SELECT ... FROM table_name;
```

Ok, let's get started.

### Getting set up

In this lesson we'll work with the northwind database, which is a sample ecommerce database.

Let's start by connecting to the database.

In [2]:
import sqlite3
conn = sqlite3.connect('Northwind_small.sqlite')
cursor = conn.cursor()

And then can see the various tables with the following.

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('Employee',),
 ('Category',),
 ('Customer',),
 ('Shipper',),
 ('Supplier',),
 ('Order',),
 ('Product',),
 ('OrderDetail',),
 ('CustomerCustomerDemo',),
 ('CustomerDemographic',),
 ('Region',),
 ('Territory',),
 ('EmployeeTerritory',)]

Now we'll only use a subset of the above tables -- focusing on the `Product`, `Supplier` and `Category` table.  Ok, so let's take a look at those tables.

> Select the first two rows from the `Product` table, the `Supplier` table, and the `Category` table to get a sense of the data.

In [5]:
import pandas as pd
query = "SELECT * FROM product LIMIT 2;" # view first two records from the Product table
pd.read_sql(query, conn)

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0


In [8]:

query = "" # view first two records from the Supplier table
pd.read_sql(, conn)

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,North America,70117,USA,(100) 555-4822,None,#CAJUN.HTM#


In [26]:
query = "" # view first two records from the Category table
pd.read_sql(query, conn)

,Id,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."


> Take a look at the columns and kind of data that we get from each table.

### Our First CTEs

Ok, now it's time to write our first CTE.  Let's use a CTE to find:
* the highest average unit price by category and supplier.

In doing so: 
* first create a temporary table called `avg_category_supplier` that computes the average unit prices per category and supplier, and 
* then find the category and supplier combination with the highest average price.

In [10]:
sql = """
WITH avg_category_supplier AS (
    SELECT CategoryId, SupplierId, AVG(UnitPrice) as avg_price FROM product GROUP BY SupplierId, CategoryId
)

SELECT * FROM avg_category_supplier 
ORDER BY avg_price desc limit 2;
"""

In [11]:
pd.read_sql(sql, conn)

# CategoryId	SupplierId	highest_avg_price
# 0	1	18	140.75

,CategoryId,SupplierId,avg_price
0,1,18,140.75
1,6,12,123.79


Now let's use a CTE to find just the category with the lowest average price.

In [16]:
sql = """
WITH avg_category AS (
    SELECT CategoryId, AVG(UnitPrice) as avg_price FROM product GROUP BY CategoryId
)

SELECT * FROM avg_category 
ORDER BY avg_price asc limit 1;
"""

In [17]:
pd.read_sql(sql, conn)

# 	CategoryId	lowest_avg_price
# 0	5	20.25

,CategoryId,avg_price
0,5,20.25


Ok, so in this section, we used CTEs to perform multiple aggregations.  We did so by using CTEs to perform an initial aggregation in a temporary table, and then queried from that temporary table.  

### CTEs for Preprocessing

Another use case for CTEs in queries is when joining together multiple tables.  Remember that in general, when coding, we often perform some initial pre-processing, and then act on that preprocessed data.  

> For example, we may just select the data we need before adding it into the database.

With CTEs, we can use temporary tables to first select just the columns that we need from a couple individual tables, and then perform the final query from there.  So we first do some initial pre-processing, and then we query.

For example, if we want to use SQL to find the different kinds of products made in the `British Isles`, we only need a few columns from the `product`, `category`, `supplier` tables.  So we'll use CTEs to first select columns from each of those individual tables, and then from there combine these temporary tables together with a JOIN. 

>Task: Use ctes to find the categories of products made in the `British Isles`.

In [28]:
sql = """
WITH select_supplier as (
    SELECT Id, CompanyName, Region FROM supplier
), 

select_category as (
    SELECT Id, CategoryName as name FROM Category
),

select_product as (
        SELECT SupplierId, CategoryId, UnitPrice FROM Product
)

SELECT name, UnitPrice, CompanyName FROM select_product 
JOIN select_supplier 
    ON select_product.SupplierId = select_supplier.Id
JOIN select_category ON select_product.CategoryId = select_category.Id
WHERE Region = 'British Isles';"""

In [29]:
pd.read_sql(sql, conn)

,name,UnitPrice,CompanyName
0,Beverages,18.0,Exotic Liquids
1,Beverages,19.0,Exotic Liquids
2,Condiments,10.0,Exotic Liquids
3,Confections,9.2,"Specialty Biscuits, Ltd."
4,Confections,81.0,"Specialty Biscuits, Ltd."
5,Confections,10.0,"Specialty Biscuits, Ltd."
6,Confections,12.5,"Specialty Biscuits, Ltd."


So we can see that the products made in the British Isles are Beverages, Condiments and Confections.

Now, take another look at the CTE, paying attention to the syntax.  Notice that there is only a single `WITH` statement, and we separate each temporary table by a comma.

```SQL
WITH select_supplier as (
    SELECT Id, CompanyName, Region FROM supplier
), 

select_category as ( ...
```

Ok, so now it's your turn to practice with using CTEs in this pattern.  Use CTEs to find the the average product price per city, and then order the cities from the highest average price to lowest.

So the first step is to use CTEs to select just the necessary columns from each of the needed tables.  And then from there we can join the tables together and perform the aggregation.

In [50]:
sql = """
WITH select_supplier as (
    SELECT Id, City FROM supplier 
),
select_product as (
    SELECT Id, SupplierId, UnitPrice FROM product 
), 
combined as (
    SELECT * FROM select_supplier 
    JOIN select_product ON 
    select_supplier.id = select_product.SupplierId
    JOIN select_category ON
    select_product.CategoryId = select_category.Id
)

SELECT City, AVG(UnitPrice) as avg_price
FROM combined GROUP BY city
ORDER BY avg_price desc
LIMIT 5
;
"""

In [49]:
pd.read_sql(sql, conn)

# 	City	avg_price
# 0	Paris	140.750
# 1	Tokyo	46.000
# 2	Frankfurt	44.678
# 3	Annecy	44.500
# 4	Ste-Hyacinthe	38.900

,City,avg_price
0,Paris,140.750
1,Tokyo,46.000
2,Frankfurt,44.678
3,Annecy,44.500
4,Ste-Hyacinthe,38.900


Next, for each category of products, find the city that has the highest average price.

In [91]:
sql = """
WITH select_supplier as (
    SELECT Id, City FROM supplier 
),
select_product as (
    SELECT Id, SupplierId, CategoryId, UnitPrice FROM product 
), 
select_category as (
    SELECT Id, CategoryName FROM Category
),
combined as (
    SELECT * FROM select_supplier 
    JOIN select_product ON 
    select_supplier.id = select_product.SupplierId
    JOIN select_category ON
    select_product.CategoryId = select_category.Id
), category_cities AS
(
 SELECT City, CategoryName, AVG(UnitPrice) as avg_price, 
 rank() over (partition by CategoryId ORDER BY AVG(UnitPrice) desc) as rank
 FROM combined GROUP BY City, CategoryId
)
SELECT * FROM category_cities WHERE rank = 1
"""

In [92]:
pd.read_sql(sql, conn)

# 	City	CategoryName	avg_price	rank
# 0	Paris	Beverages	140.75	1
# 1	Melbourne	Condiments	43.90	1
# 2	Ste-Hyacinthe	Confections	49.30	1
# 3	Annecy	Dairy Products	44.50	1
# 4	Frankfurt	Grains/Cereals	33.25	1
# 5	Frankfurt	Meat/Poultry	123.79	1
# 6	Sydney	Produce	53.00	1
# 7	Melbourne	Seafood	62.50	1

,City,CategoryName,avg_price,rank
0,Paris,Beverages,140.75,1
1,Melbourne,Condiments,43.90,1
2,Ste-Hyacinthe,Confections,49.30,1
3,Annecy,Dairy Products,44.50,1
4,Frankfurt,Grains/Cereals,33.25,1
5,Frankfurt,Meat/Poultry,123.79,1
6,Sydney,Produce,53.00,1
7,Melbourne,Seafood,62.50,1


### Summary

In this lesson we practiced using CTEs.  We use CTEs to create one or more temporary tables, and we then query from those tables.  We saw two use cases for CTEs.  With the first one, we use CTEs when chaining aggregate queries.

And with the second one, we used the temporary tables for a sort of pre-processing on each table, before then joining these tables together.  We separated each of the temporary tables with a comma.

```sql
WITH select_supplier as (
    SELECT Id, CompanyName, Region FROM supplier
), 

select_category as ( ...
```